In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '1'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'milan'
sub_set_name = 'internet'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.211626,0.212107,0.212617,0.211824,0.214977,0.212617,0.212617,0.212617,0.212617,0.258763,...,0.454009,0.503994,0.210340,0.178121,0.333928,0.304071,0.200537,0.214758,0.304316,0.208500
2013-11-01 01:00:00,0.116451,0.117075,0.117736,0.116424,0.118392,0.117736,0.117736,0.117736,0.117736,0.157255,...,0.286118,0.337854,0.179074,0.150135,0.184403,0.117824,0.065346,0.071140,0.170086,0.224719
2013-11-01 02:00:00,0.096798,0.097592,0.098434,0.096314,0.097301,0.098434,0.098434,0.098434,0.098434,0.132079,...,0.242083,0.273059,0.120781,0.093111,0.143985,0.131274,0.094235,0.097979,0.128875,0.127521
2013-11-01 03:00:00,0.040150,0.040620,0.041117,0.040729,0.041210,0.041117,0.041117,0.041117,0.041117,0.067451,...,0.219951,0.245818,0.101754,0.068147,0.068547,0.044675,0.030236,0.030274,0.060521,0.120066
2013-11-01 04:00:00,0.028765,0.029398,0.030070,0.028888,0.028997,0.030070,0.030070,0.030070,0.030070,0.055475,...,0.170191,0.217133,0.154831,0.154876,0.175142,0.126580,0.092381,0.095470,0.141605,0.165808


In [7]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.211626,0.212107,0.212617,0.211824,0.214977,0.212617,0.212617,0.212617,0.212617,0.258763,...,0.454009,0.503994,0.210340,0.178121,0.333928,0.304071,0.200537,0.214758,0.304316,0.208500
2013-11-01 01:00:00,0.116451,0.117075,0.117736,0.116424,0.118392,0.117736,0.117736,0.117736,0.117736,0.157255,...,0.286118,0.337854,0.179074,0.150135,0.184403,0.117824,0.065346,0.071140,0.170086,0.224719
2013-11-01 02:00:00,0.096798,0.097592,0.098434,0.096314,0.097301,0.098434,0.098434,0.098434,0.098434,0.132079,...,0.242083,0.273059,0.120781,0.093111,0.143985,0.131274,0.094235,0.097979,0.128875,0.127521
2013-11-01 03:00:00,0.040150,0.040620,0.041117,0.040729,0.041210,0.041117,0.041117,0.041117,0.041117,0.067451,...,0.219951,0.245818,0.101754,0.068147,0.068547,0.044675,0.030236,0.030274,0.060521,0.120066
2013-11-01 04:00:00,0.028765,0.029398,0.030070,0.028888,0.028997,0.030070,0.030070,0.030070,0.030070,0.055475,...,0.170191,0.217133,0.154831,0.154876,0.175142,0.126580,0.092381,0.095470,0.141605,0.165808


In [8]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path='./dataset/milan/'
args.data_path='internet.csv'
args.data_path

'internet.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data[0][0].shape

(96, 10000)

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
      

# Setup WANDB

In [14]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Milan Internet dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 2.9223687648773193
Epoch: 2 cost time: 2.823718309402466
Epoch: 3 cost time: 2.880047082901001
Epoch: 4 cost time: 3.231069803237915
Epoch: 5 cost time: 3.551377534866333
Epoch: 1 cost time: 3.3303539752960205
Epoch: 2 cost time: 3.308098316192627
Epoch: 3 cost time: 3.244893789291382
Epoch: 4 cost time: 3.2608113288879395
Epoch: 5 cost time: 3.284633159637451


Progress:   2%|▏         | 1/50 [00:49<40:09, 49.18s/it]

Epoch: 1 cost time: 3.565260887145996
Epoch: 2 cost time: 3.4237890243530273
Epoch: 3 cost time: 2.9332945346832275
Epoch: 4 cost time: 2.8744475841522217
Epoch: 5 cost time: 2.5942389965057373
Epoch: 1 cost time: 2.634498119354248
Epoch: 2 cost time: 2.807541608810425
Epoch: 3 cost time: 2.7259631156921387
Epoch: 4 cost time: 2.737649917602539
Epoch: 5 cost time: 2.6219494342803955


Progress:   4%|▍         | 2/50 [01:33<37:13, 46.52s/it]

...

Progress:  96%|█████████▌| 48/50 [36:09<01:31, 45.88s/it]

Epoch: 1 cost time: 2.7150840759277344
Epoch: 2 cost time: 2.7350668907165527
Epoch: 3 cost time: 2.752040147781372
Epoch: 4 cost time: 2.618532419204712
Epoch: 5 cost time: 2.6865949630737305
Epoch: 1 cost time: 2.5649943351745605
Epoch: 2 cost time: 2.607041835784912
Epoch: 3 cost time: 2.7102508544921875
Epoch: 4 cost time: 2.8107006549835205
Epoch: 5 cost time: 3.126596450805664


Progress:  98%|█████████▊| 49/50 [36:54<00:45, 45.63s/it]

Epoch: 1 cost time: 2.597545623779297
Epoch: 2 cost time: 2.6431353092193604
Epoch: 3 cost time: 2.7455830574035645
Epoch: 4 cost time: 2.7292542457580566
Epoch: 5 cost time: 2.5640883445739746
Epoch: 1 cost time: 2.587702512741089
Epoch: 2 cost time: 2.505488395690918
Epoch: 3 cost time: 2.8555119037628174
Epoch: 4 cost time: 3.143329620361328
Epoch: 5 cost time: 3.383490562438965


Progress: 100%|██████████| 50/50 [37:39<00:00, 45.18s/it]
